In [4]:
%reload_ext autoreload
%autoreload 2

from dev.dec import load_metadata, organize_jobs_by_calibration_date, retrieve_calib_data, retrieve_memories_and_job_info, get_pSoft_and_countMat
from result_saver import SaverProvider

provider = SaverProvider()

In [ ]:
from soft_info import get_repcode_IQ_map, inv_qubit_mapping
from Scratch import load_calibration_memory
import cpp_soft_info as csi
from datetime import datetime
import pickle


DEVICE = 'ibm_sherbrooke'
LOGICAL = '1'
XBASIS = False
ROUNDS = 100

for LOGICAL in ['0', '1']:
    for XBASIS in [True, False]:
        md = load_metadata(DEVICE, LOGICAL, XBASIS, ROUNDS)
        jobs_by_calib_date = organize_jobs_by_calibration_date(md)
        
        pSofts = []
        countMats = []
        estim0Mats = []
        estim1Mats = []
        for idx, calib_date in enumerate(jobs_by_calib_date.keys()):
            if idx > 1:
                break
            job_ids = jobs_by_calib_date[calib_date]
            all_memories_PS, _, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                                other_date=calib_date, post_process=True,
                                                                                double_msmt=True)

            kde_dict_PS = csi.get_KDEs(all_memories_PS, [0.6], relError=1, absError=-1, num_points=20)
            big_memory, d, T, link_qubits, code_qubits, shots_per_job = retrieve_memories_and_job_info(provider, job_ids, md)
            inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(link_qubits+code_qubits, T))

            print(f"\nstarting {calib_date} {DEVICE} {LOGICAL} {XBASIS} at {datetime.now()}")
            pSoft_PS, countMat_PS, estim0MatPS, estim1MatPS = csi.iqConvertorEstim(big_memory, inverted_q_map, kde_dict_PS, 1, -1)
            print(f"done {calib_date} {DEVICE} {LOGICAL} {XBASIS} at {datetime.now()}\n")

            pSofts.append(pSoft_PS)
            countMats.append(countMat_PS)
            estim0Mats.append(estim0MatPS)
            estim1Mats.append(estim1MatPS)
        
        file_name = f"./results/Leakage/{DEVICE}_{LOGICAL}_{XBASIS}_{ROUNDS}.pkl"
        with open(file_name, 'wb') as f:
            pickle.dump((pSofts, countMats, estim0Mats, estim1Mats), f)
        print(f"saved to {file_name}")



In [7]:
from soft_info import get_repcode_IQ_map, inv_qubit_mapping
from Scratch import load_calibration_memory
import cpp_soft_info as csi
from datetime import datetime
import pickle


DEVICE = 'ibm_sherbrooke'
LOGICAL = '1'
XBASIS = False
ROUNDS = 100

for LOGICAL in ['0', '1']:
    for XBASIS in [True, False]:
        md = load_metadata(DEVICE, LOGICAL, XBASIS, ROUNDS)
        jobs_by_calib_date = organize_jobs_by_calibration_date(md)
        
        with open(f"./results/Leakage/{DEVICE}_{LOGICAL}_{XBASIS}_{ROUNDS}.pkl", 'rb') as f:
            (pSofts, countMats, estim0Mats, estim1Mats) = pickle.load(f)

        inverted_q_maps = []
        big_memories = []
        for idx, calib_date in enumerate(jobs_by_calib_date.keys()):
            if idx > 1:
                break
            job_ids = jobs_by_calib_date[calib_date]

            big_memory, d, T, link_qubits, code_qubits, shots_per_job = retrieve_memories_and_job_info(provider, job_ids, md)
            inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(link_qubits+code_qubits, T))

            inverted_q_maps.append(inverted_q_map)
            big_memories.append(big_memory)
        
        file_name = f"./results/Leakage/Correct_{DEVICE}_{LOGICAL}_{XBASIS}_{ROUNDS}.pkl"
        with open(file_name, 'wb') as f:
            pickle.dump((pSofts, countMats, estim0Mats, estim1Mats, inverted_q_maps, big_memories), f)
        print(f"saved to {file_name}")



shape md before: (40, 18)
State: X0 100
shape md: (20, 18)
Calibration dates: dict_keys([datetime.datetime(2024, 3, 23, 16, 56, 36, 242000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 3, 23, 16, 51, 21, 688000, tzinfo=datetime.timezone.utc)])
Num of calibrations: 2
Num of jobs per calibration: [11, 9]

saved to ./results/Leakage/Correct_ibm_sherbrooke_0_True_100.pkl
shape md before: (40, 18)
State: Z0 100
shape md: (20, 18)
Calibration dates: dict_keys([datetime.datetime(2024, 3, 23, 16, 44, 31, 639000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 3, 23, 16, 38, 32, 429000, tzinfo=datetime.timezone.utc)])
Num of calibrations: 2
Num of jobs per calibration: [11, 9]

saved to ./results/Leakage/Correct_ibm_sherbrooke_0_False_100.pkl
shape md before: (40, 18)
State: X1 100
shape md: (20, 18)
Calibration dates: dict_keys([datetime.datetime(2024, 3, 23, 17, 2, 9, 643000, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 3, 23, 16, 56, 49, 544000, tzinfo=datetime.tim

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from soft_info import plot_IQ_data_with_countMat

d = 52
T = 100
# select the data from round to round 
def get_cols_of_round(round_num, d, T):
    '''Indexing starts at 0'''
    assert round_num < T
    return list(range(round_num*(d-1), (round_num+1)*(d-1)))

n_leaked = []
n_leaked_PS = []
threshold = 0.01
for round in range(T):

    # if round > 5:
    #     continue

    round_cols = get_cols_of_round(round, d, T)
    # pSoft_round = pSoft[:, round_cols]
    pSoft_PS_round = pSoft_PS[:, round_cols]
    # estim0Mat_round = estim0Mat[:, round_cols]
    # estim1Mat_round = estim1Mat[:, round_cols]
    estim0MatPS_round = estim0MatPS[:, round_cols]
    estim1MatPS_round = estim1MatPS[:, round_cols]

    # mask_round = ((estim0Mat_round < threshold) & (estim1Mat_round < threshold))  
    mask_PS_round = ((estim0MatPS_round < threshold) & (estim1MatPS_round < threshold))  

    IQ_data = big_memory[:, round_cols]

    col_idx = 0
    # mask_col = mask_round[:, col_idx]
    mask_PS_col = mask_PS_round[:, col_idx]
    IQ_data_col = IQ_data[:, col_idx]
    # countMat_col = countMat[:, col_idx]
    countMat_PS_col = countMat_PS[:, col_idx]   
    # plot_IQ_data_with_countMat(IQ_data_col[mask_col], countMat_col[mask_col], title=f'Round {round} - pSoft')
    # plot_IQ_data_with_countMat(IQ_data_col[mask_PS_col], countMat_PS_col[mask_PS_col], title=f'Round {round} - pSoft_PS')

    # how many times the mask is true
    # print(f"Round {round} - mask: {mask_round.sum()} - mask_PS: {mask_PS_round.sum()} out of {pSoft_PS_round.shape[0]*pSoft_PS_round.shape[1]}")
    # n_leaked.append(mask_round.sum()/mask_round.size)
    n_leaked_PS.append(mask_PS_round.sum()/mask_PS_round.size)

n_leaked = np.array(n_leaked)
n_leaked_PS = np.array(n_leaked_PS)

# plt.plot(n_leaked-n_leaked[0], label='pSoft')
plt.plot(n_leaked_PS-n_leaked_PS[0], label='pSoft_PS')
plt.xlabel('Round')
plt.xticks(range(0, T, 10))
plt.ylabel('Leakage ratio')
plt.grid(which='both')
plt.legend()
plt.title(f"Leakage ratio difference between rounds {DEVICE} logical: {LOGICAL} xbasis:{XBASIS} ")
